In [1]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

import pandasql as psql

# pip install pandasql

In [2]:
# Load the Universal bank data

loans = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\44 season 17-jul-2021\loan_data.csv", header=0) 
loans.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [3]:
# Identify the category variables

cat_cols = ['purpose']

# Convert the catagory variable into dummy variables

loans = pd.get_dummies(loans,columns=cat_cols)
loans=pd.DataFrame(loans)
loans.head()

,credit.policy,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid,purpose_all_other,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_major_purchase,purpose_small_business
0,1,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0,0,0,1,0,0,0,0
1,1,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0,0,1,0,0,0,0,0
2,1,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0,0,0,1,0,0,0,0
3,1,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0,0,0,1,0,0,0,0
4,1,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0,0,1,0,0,0,0,0


In [4]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   credit.policy               9578 non-null   int64  
 1   int.rate                    9578 non-null   float64
 2   installment                 9578 non-null   float64
 3   log.annual.inc              9578 non-null   float64
 4   dti                         9578 non-null   float64
 5   fico                        9578 non-null   int64  
 6   days.with.cr.line           9578 non-null   float64
 7   revol.bal                   9578 non-null   int64  
 8   revol.util                  9578 non-null   float64
 9   inq.last.6mths              9578 non-null   int64  
 10  delinq.2yrs                 9578 non-null   int64  
 11  pub.rec                     9578 non-null   int64  
 12  not.fully.paid              9578 non-null   int64  
 13  purpose_all_other           9578 

In [14]:
loans['credit.policy'].value_counts()

1    7710
0    1868
Name: credit.policy, dtype: int64

In [7]:
# Identify the dependent and Target variables

IndepVar = []
for col in loans.columns:
    if col != 'credit.policy':
        IndepVar.append(col)

TargetVar = 'credit.policy'

x = loans[IndepVar]
y = loans[TargetVar]

In [8]:
# Splitting the dataset into train and test 

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 12)
x_test_F1 = x_test.copy()

In [9]:
# Identify variables which are fit for scaling

cols = ['int.rate', 'installment', 'log.annual.inc', 'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util']

# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols] = mmscaler.fit_transform(x_train[cols])
x_train = pd.DataFrame(x_train)

x_test[cols] = mmscaler.fit_transform(x_test[cols])
x_test = pd.DataFrame(x_test)

# Random forest without LDA/PCA

In [10]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
criterion='entropy', max_depth=2, max_features='auto',
max_leaf_nodes=None, max_samples=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=500,
n_jobs=None, oob_score=False, random_state=0, verbose=0,
warm_start=False)

modelRF = modelRF.fit(x_train, y_train)

# Predict the model with test data set

y1_pred = modelRF.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y1_pred))
print(classification_report(y_test, y1_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y1_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y1_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y1_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y1_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3))

[[ 121  464]
 [   0 2289]]
              precision    recall  f1-score   support

           0       1.00      0.21      0.34       585
           1       0.83      1.00      0.91      2289

    accuracy                           0.84      2874
   macro avg       0.92      0.60      0.63      2874
weighted avg       0.87      0.84      0.79      2874

Accuracy: 83.86 %
Precision: 83.86 %
Recall: 83.86 %
f1-score: 83.86 %
roc_auc_score: 0.603


# Decision Tree without LDA/PCA

In [11]:
# To build the decision tree model with Over sampling

from sklearn.tree import DecisionTreeClassifier

modelDT = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
max_depth=None, max_features=None, max_leaf_nodes=None,
min_impurity_decrease=0.0, min_impurity_split=None,
min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
random_state=None, splitter='best')

modelDT = modelDT.fit(x_train,y_train)

# Predict with test data

y3_pred = modelDT.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y3_pred))
print(classification_report(y_test, y3_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y3_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y3_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y3_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y3_pred) * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3))

[[ 570   15]
 [ 237 2052]]
              precision    recall  f1-score   support

           0       0.71      0.97      0.82       585
           1       0.99      0.90      0.94      2289

    accuracy                           0.91      2874
   macro avg       0.85      0.94      0.88      2874
weighted avg       0.93      0.91      0.92      2874

Accuracy: 91.23 %
Precision: 99.27 %
Recall: 89.65 %
f1-score: 94.21 %
roc_auc_score: 0.935


# Logistic Regression without LDA/PCA

In [12]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

modelLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
intercept_scaling=1, max_iter=100, multi_class='auto',
n_jobs=None, penalty='l2', random_state=None,
solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

modelLR = modelLR.fit(x_train,y_train)

# Predict the model with test data set

y5_pred = modelLR.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y5_pred))
print(classification_report(y_test, y5_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y5_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y5_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y5_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y5_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve
print('roc_auc_score:', round(roc_auc_score(y_test, y5_pred), 3))

[[ 365  220]
 [  71 2218]]
              precision    recall  f1-score   support

           0       0.84      0.62      0.71       585
           1       0.91      0.97      0.94      2289

    accuracy                           0.90      2874
   macro avg       0.87      0.80      0.83      2874
weighted avg       0.89      0.90      0.89      2874

Accuracy: 89.87 %
Precision: 89.87 %
Recall: 89.87 %
f1-score: 89.87 %
roc_auc_score: 0.796


# SVM - Gaussian without LDA/PCA 

In [13]:
# Training the SVM algorithm

from sklearn.svm import SVC

modelSVMGaussian = SVC(kernel='rbf', random_state = None, class_weight=None,probability=True)
modelSVMGaussian.fit(x_train, y_train)

# Predicting the values

y_pred3 = modelSVMGaussian.predict(x_test)

# Confusion matrix and classification report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test,y_pred3))
print(classification_report(y_test,y_pred3))

# Evalution metrics

import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, roc_auc_score

print("Accuracy:", (round(metrics.accuracy_score(y_test, y_pred3) * 100, 2)), "%")
print("Precision:", (round(metrics.precision_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print("Recall:", (round(metrics.recall_score(y_test, y_pred3, average='micro') * 100, 2)), "%")
print("f1-score:", (round(metrics.f1_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('roc_auc_score:', round(roc_auc_score(y_test, y_pred3), 3))

[[ 345  240]
 [  33 2256]]
              precision    recall  f1-score   support

           0       0.91      0.59      0.72       585
           1       0.90      0.99      0.94      2289

    accuracy                           0.91      2874
   macro avg       0.91      0.79      0.83      2874
weighted avg       0.91      0.91      0.90      2874

Accuracy: 90.5 %
Precision: 90.5 %
Recall: 90.5 %
f1-score: 90.5 %
roc_auc_score: 0.788


# Appling PCA to all 5 Algorithm 

In [ ]:
from sklearn.decomposition import PCA

applyPCA = PCA()

x_train = applyPCA.fit_transform(x_train)
x_test = applyPCA.transform(x_test)
explained_variance = applyPCA.explained_variance_ratio_
print(explained_variance)

In [ ]:
# Random forest with 'Linear Discriminant Analysis' technique to reduce the dimentionality

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

applyLDA = LinearDiscriminantAnalysis()

x_train = applyLDA.fit_transform(x_train, y_train)
x_test = applyLDA.transform(x_test)

explained_variance = applyLDA.explained_variance_ratio_
print(explained_variance)